# Exploring and creating cluster for the neighborhoods in Toronto.


&emsp;In this assignment,we explore, segment, and cluster the neighborhoods in the city of Toronto.For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We are required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

&emsp;Once the data is in a structured format, we replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.



## Importing required libraries for analysis


In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Importing data from wikipedia using request function and re-assinging it to a variable name.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')


In [3]:
class HTMLTableParser:
       
        def parse_url(self, path):
            response = requests.get(path)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.parse_html_table(table))\
                    for table in soup.find_all('table', class_="wikitable sortable")]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [4]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
hp = HTMLTableParser()
table = hp.parse_url(path)[0] # Grabbing the table from the tuple
table.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


# Creating a variable name table with only the assigned values

In [5]:
table = table[table.Borough != 'Not assigned']
table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [6]:

list(table)

['Postcode', 'Borough', 'Neighbourhood\n']

# Replacing all the '\n' with ' '

In [7]:
table = table.replace('\n',' ', regex=True)
table.head(10)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [8]:
table = table[table['Neighbourhood\n'] != 'Not assigned']
table.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [9]:

df = table.groupby(['Postcode','Borough'])['Neighbourhood\n'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,"Adelaide , King , Richmond"
1,M3H,North York,"Bathurst Manor , Downsview North , Wilson Heig..."
2,M9L,North York,Humber Summit
3,M6C,York,Humewood-Cedarvale
4,M6J,West Toronto,"Little Portugal , Trinity"
5,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
6,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
7,M3K,North York,"CFB Toronto , Downsview East"
8,M2N,North York,Willowdale South
9,M3M,North York,Downsview Central


In [10]:
df.shape

(103, 3)

In [11]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
list(df)

['Postcode', 'Borough', 'Neighbourhood\n']

In [13]:

list(geo_data)

['Postal Code', 'Latitude', 'Longitude']

# Combination table 

In [14]:
table = df.set_index('Postcode').join(geo_data.set_index('Postal Code'))
table = table.sample(frac=1).reset_index(drop=True)
table.head(20)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Willowdale West,43.782736,-79.442259
1,East Toronto,Studio District,43.659526,-79.340923
2,Central Toronto,"Deer Park , Forest Hill SE , Rathnelly , South...",43.686412,-79.400049
3,York,Caledonia-Fairbanks,43.689026,-79.453512
4,Scarborough,Upper Rouge,43.836125,-79.205636
5,Etobicoke,"Albion Gardens , Beaumond Heights , Humbergate...",43.739416,-79.588437
6,West Toronto,"Brockton , Exhibition Place , Parkdale Village",43.636847,-79.428191
7,North York,Parkwoods,43.753259,-79.329656
8,North York,Hillcrest Village,43.803762,-79.363452
9,Scarborough,"Clarks Corners , Sullivan , Tam O'Shanter",43.781638,-79.304302


In [16]:

table.shape

(103, 4)